<a href="https://colab.research.google.com/github/prabinbohara10/Fine-tuning-LLMs/blob/master/LoRA/LoRA_Finetuning_huggingface_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and importing libraries

In [ ]:
!pip install pytorch

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)


In [1]:
!pip install transformers datasets peft evaluate bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch

import numpy as np
import pandas as pd

import os

In [4]:
# mount drive and create symlink
from google.colab import drive
drive.mount('/content/drive')
# make sure to check if this is the correct directory for your Google account some people do not have a space in 'My Drive'
!ln -s "/content/drive/My Drive/nlp-datasets/text-classification" "/content/text-classification" # NB when changing this

Mounted at /content/drive


In [5]:
data_folder = "text-classification"
train_csv_path = os.path.join(data_folder, "csmb_preprocessed/train_aug_preprocessed_0.csv")
#test_csv_path = os.path.join(data_folder, "test.csv")

## Loading the csv and preprocess

In [90]:
df = pd.read_csv(train_csv_path)
df

,id,dataset,text,toxicity,sexist,of_id
0,1364,other,"*thinks about every war in history* No, men ne...",0.346308,False,-1
1,1365,callme,a game with a female as main character sucks,0.899755,True,-1
2,1366,other,shhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh MKR — watchi...,0.207430,False,-1
3,1367,other,MENTION1981 pravana vivids are my favorite lin...,0.096136,False,-1
4,1368,other,Cc MENTION3515 [LINK],0.085468,False,-1
...,...,...,...,...,...,...
12262,13630,callme,this reminds me of the MENTION3079 situation; ...,0.147044,False,-1
12263,13631,other,mkr I love Annie and loyld there like a real l...,0.213106,False,-1
12264,13632,other,No u. [LINK],0.324702,False,-1
12265,13633,other,mkr the way kat looks at Annie is like she's s...,0.563036,False,-1


In [91]:
df['label'] = df['sexist'].astype('category')
df['target'] = df['label'].cat.codes
df

,id,dataset,text,toxicity,sexist,of_id,label,target
0,1364,other,"*thinks about every war in history* No, men ne...",0.346308,False,-1,False,0
1,1365,callme,a game with a female as main character sucks,0.899755,True,-1,True,1
2,1366,other,shhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh MKR — watchi...,0.207430,False,-1,False,0
3,1367,other,MENTION1981 pravana vivids are my favorite lin...,0.096136,False,-1,False,0
4,1368,other,Cc MENTION3515 [LINK],0.085468,False,-1,False,0
...,...,...,...,...,...,...,...,...
12262,13630,callme,this reminds me of the MENTION3079 situation; ...,0.147044,False,-1,False,0
12263,13631,other,mkr I love Annie and loyld there like a real l...,0.213106,False,-1,False,0
12264,13632,other,No u. [LINK],0.324702,False,-1,False,0
12265,13633,other,mkr the way kat looks at Annie is like she's s...,0.563036,False,-1,False,0


In [92]:
df['label'].cat.categories

Index([False, True], dtype='bool')

In [93]:
category_map = {code: category for code, category in enumerate(df['label'].cat.categories)}
category_map

{0: False, 1: True}

In [94]:
# define label maps
id2label = category_map
label2id ={v: k for k, v in id2label.items()}
label2id

{False: 0, True: 1}

In [95]:
df = df[['text', 'target']]
df.columns = ['text', 'label']
df

,text,label
0,"*thinks about every war in history* No, men ne...",0
1,a game with a female as main character sucks,1
2,shhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh MKR — watchi...,0
3,MENTION1981 pravana vivids are my favorite lin...,0
4,Cc MENTION3515 [LINK],0
...,...,...
12262,this reminds me of the MENTION3079 situation; ...,0
12263,mkr I love Annie and loyld there like a real l...,0
12264,No u. [LINK],0
12265,mkr the way kat looks at Annie is like she's s...,0


In [96]:
train_end_point = int(df.shape[0]*0.6)
val_end_point = int(df.shape[0]*0.8)
df_train = df.iloc[:train_end_point,:]
df_val = df.iloc[train_end_point:val_end_point,:]
df_test = df.iloc[val_end_point:,:]
print(df_train.shape, df_test.shape, df_val.shape)

(7360, 2) (2454, 2) (2453, 2)


## Pandas df to Hugging Face dataset

In [97]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

In [98]:
# Shuffle the training dataset
dataset_train_shuffled = dataset_train.shuffle(seed=42)  # Using a seed for reproducibility

In [99]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train_shuffled,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7360
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 2453
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2454
    })
})

In [100]:
df_train.label.value_counts(normalize=True)

,proportion
label,
0,0.866033
1,0.133967


In [101]:
# display % of training data with label=1
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.13396739130434782

## Load base model

In [102]:
# model name:
model_name = "distilbert-base-uncased"

In [103]:
# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [105]:
# Display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Preprocess Data: Tokenization and padding

In [106]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [107]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [108]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/7360 [00:00<?, ? examples/s]

Map:   0%|          | 0/2453 [00:00<?, ? examples/s]

Map:   0%|          | 0/2454 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7360
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2453
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2454
    })
})

In [109]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluation

In [110]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [111]:
# define an ev inaluation function to passto trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

## Apply untrained model to text

In [112]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + str(id2label[predictions.tolist()]))

Untrained model predictions:
----------------------------
It was good. - True
Not a fan, don't recommed. - True
Better than the first one. - True
This is not worth watching even once. - True
This one is a pass. - True


## Get model with peft config

In [113]:
# LoRA configuration:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [114]:
# #Optional if quantization is needed:
# # quantization config:
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit = True, # enable 4-bit quantization
#     bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
#     bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
#     bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
# )

In [115]:
# get model with lora configuration
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


## Training parameters: (hyperparametes, training arguments)

In [116]:
# hyperparameters:
lr = 1e-3
batch_size = 4
num_epochs = 3

In [117]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_name + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Trainer Object and model training

In [118]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

In [119]:
# train model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 